# Preprocessing
## 6. Sampling data for modelling

- In this file we create the valiation dataset for each setup described in the thesis. The table below shows the folder names and the setups that were created.

| id | Folder name        | Setup             | Description                                                                                                                      |
|----|--------------------|-------------------|----------------------------------------------------------------------------------------------------------------------------------|
| 1  | allpredictors      | -                 | Folder containing allpredictors tables for all stations within the study area. Variables include GHMs outputs, meteorological variables and observed discharge.                                                                                                               |
| 2  | allpredictors_pcr  | -                 | Folder containing PCR_allpredictors table from Magni et al. (2023) dataset. All stations with the study area.                                                                                       |
| 3  | all_predCachAtt    | -                 | Folder containing data from allpredictors and catchment attributes from Magni et al. (2023) dataset. All stations with the study area.                                                               |
| 4  | all_stations       | all_stations      | Cross-validation dataset of all_stations setup containing 5 subsamples of all the dataset within the study area.                                                                          |
| 5  | elbe               | rhine_elbe        | Folder containing training/validation data for rhine_elbe setup.                                                                  |
| 6  | elbe_catch         | rhine_elbe_catch  | Folder containing training/validation data for rhine_elbe setup with catchment attributes.                                       |
| 7  | maas               | rhine_maas        | Folder containing training/validation data for rhine_maas setup.                                                                  |
| 8  | maas_catch         | rhine_maas_catch  | Folder containing training/validation data for rhine_maas setup with catchment attributes.                                       |
| 9  | rhine_catch        | rhine_only_catch  | Cross-validation dataset of rhine_only setup with catchment attributes containing 5 subsamples of all stations within rhine basin. |
| 10 | rhine_only         | allpredictors_catch | Cross-validation dataset of rhine_only setup containing 5 subsamples of all stations within rhine basin.                            |
| 11 | rhine_pcr          | PCR_allpredictors | Cross-validation dataset of rhine_only setup with catchment attributes containing 5 subsamples of all stations within rhine basin. |


**Download folder: raw_data data that contains results from all the previous pre-processing steps. We sample from the following folders:**

    - allpredictors
    - allpredictors_pcr
    - allpredCatchAtt

In [1]:
import os
import shutil
import random
import pandas as pd
import shutil

In [2]:
setup_dict = {
    1: "all_stations",
    2: "elbe",
    3: "elbe_catch",
    4: "maas",
    5: "maas_catch",
    6: "rhine_catch",
    7: "rhine_only",
    8: "rhine_pcr"
}

In [3]:
seed_value = 123 
random.seed(seed_value)

In [4]:
def normalize_dis(column, area):
    time = 24 * 3600
    area_m = area*1000000
    new_column = column * (time / area_m)
    return new_column


def normalize_columns_with_dis(df, area):
    for column in df.columns:
        if 'dis' in column:
            df[column] = normalize_dis(df[column], area)
    return df

In [5]:
def subsample_table(stations, filenames, area):
    grdc_nos = [str(grdc_no) for grdc_no in stations['grdc_no']]
    sub_filenames = [filename for filename in filenames if any(grdc_no in filename for grdc_no in grdc_nos)]
    sub_datas = []
    
    
    for filename in sub_filenames:
        sub_data = pd.read_csv(filename)
        
        # Convert 'datetime' column to datetime type
        sub_data['datetime'] = pd.to_datetime(sub_data['datetime'])
        
        # Subset the data based on the datetime range
        start_date = pd.to_datetime('1979-01-01')
        end_date = pd.to_datetime('2012-12-31')
        sub_data = sub_data[(sub_data['datetime'] >= start_date) & (sub_data['datetime'] <= end_date)]
        
        
        sub_datas.append(sub_data)
    
    sub_table = pd.concat(sub_datas, ignore_index=True)
    
    return sub_table

In [55]:
def get_file_path_preds(setup_name):
    if setup_name:
        if 'catch' in setup_name:
            file_path_preds = '../raw_data/all_predCachAtt/'
        elif 'pcr' in setup_name:
            file_path_preds = '../raw_data/allpredictors_pcr/'
        else:
            file_path_preds = '../raw_data/allpredictors/' 
            
        return file_path_preds
        print(f"Setup ID: {setup_id}, Setup Name: {setup_name}, File Path Preds: {file_path_preds}")
    else:
        print(f"Invalid Setup ID: {setup_id}")


setup_dict = {
    1: "all_stations",
    2: "elbe",
    3: "elbe_catch",
    4: "maas",
    5: "maas_catch",
    6: "rhine_catch",
    7: "rhine_only",
    8: "rhine_pcr"
}


In [7]:
station_info = pd.read_csv("../../../data/stations_rhine_elbe.csv")
rhine_stations = station_info.loc[(station_info["sub_reg"] == 6351) | (station_info["sub_reg"] == 6361)]
elbe_stations = station_info.loc[(station_info["sub_reg"] == 6401)]
maas_stations = station_info.loc[(station_info["sub_reg"] == 6211)]

In [8]:
elbe_stations = elbe_stations[~elbe_stations["grdc_no"].isin([6340300, 6340301])]

# Rhine subsampling

In [50]:
setup_rhine = setup_dict[8] # 6, 7 or 8
setup_rhine

'rhine_pcr'

In [51]:
# File paths
file_path_preds = get_file_path_preds(setup_rhine)
print(file_path_preds)
file_list_preds = os.listdir(file_path_preds)
file_paths = [os.path.join(file_path_preds, file) for file in file_list_preds]

../../../data/allpredictors_pcr/


In [52]:
list_ids_rhine = rhine_stations["grdc_no"].to_list()
list_ids_all = station_info['grdc_no'].to_list()

station_info_rhine = station_info[station_info["grdc_no"].isin([x for x in list_ids_rhine])]

In [53]:
output_base_dir = f'../R/data/{setup_rhine}/'
os.makedirs(output_base_dir, exist_ok=True)

sample_number = 21

for subsample in range(1, 6):
    output_dir = os.path.join(output_base_dir, f'subsample_{subsample}')
    os.makedirs(output_dir, exist_ok=True)
    
    print(f'Sampling Subsample {subsample}...')
    
    ## Subset train stations randomly:
    train_station_ids = random.sample(list_ids_rhine, sample_number)
    train_stations = station_info_rhine[station_info_rhine['grdc_no'].isin([x for x in train_station_ids])]
    
    # Subset test stations
    test_stations =  station_info_rhine[~station_info_rhine["grdc_no"].isin([x for x in train_station_ids])]
    
    
    # Create train table
    train_table = subsample_table(train_stations, file_paths, station_info)
    train_table['datetime'] = pd.to_datetime(train_table['datetime']).dt.date
    
    # Create train test
    test_table = subsample_table(test_stations, file_paths,station_info)
    test_table['datetime'] = pd.to_datetime(test_table['datetime']).dt.date
    

    nrow_train = train_table.shape[0]
    nrow_test = test_table.shape[0]
    
    ratio_subsamples = nrow_train / (nrow_train + nrow_test)
    
    print(ratio_subsamples)
    
    
    # Write tables: train_stations, test_stations, train_table
    train_stations.to_csv(os.path.join(output_dir, 'train_stations.csv'), index=False)
    test_stations.to_csv(os.path.join(output_dir, 'test_stations.csv'), index=False)
    train_table.to_csv(os.path.join(output_dir, 'train_table_allpredictors.csv'), index=False)
    test_table.to_csv(os.path.join(output_dir, 'test_table_allpredictors.csv'), index=False)



    print(f'Finished Subsample {subsample}...')

Sampling Subsample 1...
0.7
Finished Subsample 1...
Sampling Subsample 2...
0.7
Finished Subsample 2...
Sampling Subsample 3...
0.7
Finished Subsample 3...
Sampling Subsample 4...
0.7
Finished Subsample 4...
Sampling Subsample 5...
0.7
Finished Subsample 5...


setup_rhine = setup_dict[6] # 6, 7 or 8
setup_rhine#station_info = pd.read_csv("../data/stations_rhine_elbe.csv")
# Random sampling all_stations (70/30)

In [29]:
setup_allstations = setup_dict[1] # 1
print(setup_allstations)
# File paths
file_path_preds = get_file_path_preds(setup_allstations)
print(file_path_preds)
file_list_preds = os.listdir(file_path_preds)
file_paths = [os.path.join(file_path_preds, file) for file in file_list_preds]

all_stations
../../../data/allpredictors/


In [30]:
# Iterate over sub-samples
# Path configurations
setup = "all_stations"

output_base_dir = f'../R/data/{setup_allstations}/'
os.makedirs(output_base_dir, exist_ok=True)

for subsample in range(1, 6):
    output_dir = os.path.join(output_base_dir, f'subsample_{subsample}')
    os.makedirs(output_dir, exist_ok=True)
    
    print(f'Sampling Subsample {subsample}...')
    
    ## Subset train stations randomly:
    train_stations = random.sample(list(station_info['grdc_no']), 35)
    train_stations = station_info[station_info['grdc_no'].isin(train_stations)]
    
    # Subset test stations
    test_stations = station_info[~station_info['grdc_no'].isin(train_stations['grdc_no'])]
    
    
    # Create train table
    train_table = subsample_table(train_stations, file_paths, station_info)
    train_table['datetime'] = pd.to_datetime(train_table['datetime']).dt.date
    
    # Create train test
    test_table = subsample_table(test_stations, file_paths,station_info)
    test_table['datetime'] = pd.to_datetime(test_table['datetime']).dt.date
    

    nrow_train = train_table.shape[0]
    nrow_test = test_table.shape[0]
    
    ratio_subsamples = nrow_train / (nrow_train + nrow_test)
    
    print(ratio_subsamples)
    
    # Sample file paths for test stations
    test_file_paths = random.sample(file_paths, k=len(test_stations))
    
    # Filter file paths for train stations
    train_file_paths = [file_path for file_path in file_paths if file_path not in test_file_paths]
    
    # Write tables: train_stations, test_stations, train_table
    train_stations.to_csv(os.path.join(output_dir, 'train_stations.csv'), index=False)
    test_stations.to_csv(os.path.join(output_dir, 'test_stations.csv'), index=False)
    train_table.to_csv(os.path.join(output_dir, 'train_table_allpredictors.csv'), index=False)
    test_table.to_csv(os.path.join(output_dir, 'test_table_allpredictors.csv'), index=False)

    # Save test file paths
    with open(os.path.join(output_dir, 'test_file_paths.txt'), 'w') as f:
        for file_path in test_file_paths:
            f.write(file_path + '\n')
    
     
    # Save train file paths
    with open(os.path.join(output_dir, 'train_file_paths.txt'), 'w') as f:
        for file_path in train_file_paths:
            f.write(file_path + '\n')

    print(f'Finished Subsample {subsample}...')

Sampling Subsample 1...
0.6756986054403733
Finished Subsample 1...
Sampling Subsample 2...
0.6950527599554589
Finished Subsample 2...
Sampling Subsample 3...
0.6859324460469802
Finished Subsample 3...
Sampling Subsample 4...
0.6979161143220743
Finished Subsample 4...
Sampling Subsample 5...
0.6879473991197836
Finished Subsample 5...


# Rhine_elbe or Rhine_maas Setups

In [43]:
# Rhine_elbe or Rhine_maas Setups

setup_elbe_maas = setup_dict[5] # elbe - 2, 3 or maas - 4, 5

print(setup_elbe_maas)
# File paths
file_path_preds = get_file_path_preds(setup_elbe_maas)
print(file_path_preds)
file_list_preds = os.listdir(file_path_preds)
file_paths = [os.path.join(file_path_preds, file) for file in file_list_preds]

maas_catch
../../../data/all_predCachAtt/


In [44]:
output_dir = f'../R/data/{setup_elbe_maas}/'
os.makedirs(output_dir, exist_ok=True)

In [45]:
#station_info[~station_info['grdc_no'].isin(test_stations['grdc_no'])]

# Create train stations
train_stations = rhine_stations

# Create train table
train_table = subsample_table(train_stations, file_paths, station_info)
train_table['datetime'] = pd.to_datetime(train_table['datetime']).dt.date

 # Create train test
test_table = subsample_table(test_stations, file_paths,station_info)
test_table['datetime'] = pd.to_datetime(test_table['datetime']).dt.date


nrow_train = train_table.shape[0]
nrow_test = test_table.shape[0]

ratio_subsamples = nrow_train / (nrow_train + nrow_test)

print(ratio_subsamples)

# Sample file paths for test stations
test_file_paths = random.sample(file_paths, k=len(test_stations))

# Filter file paths for train stations
train_file_paths = [file_path for file_path in file_paths if file_path not in test_file_paths]

# Write tables: train_stations, test_stations, train_table
train_stations.to_csv(os.path.join(output_dir, 'train_stations.csv'), index=False)
test_stations.to_csv(os.path.join(output_dir, 'test_stations.csv'), index=False)
train_table.to_csv(os.path.join(output_dir, 'train_table_allpredictors.csv'), index=False)
test_table.to_csv(os.path.join(output_dir, 'test_table_allpredictors.csv'), index=False)

0.6653017118807939


# All the pre-processed files can be found here: https://zenodo.org/record/8092323